In [2]:
import json

# Training Data
trainLabels = {}
with open("train.label.json",  encoding="utf8") as f:
    trainLabels = json.load(f)    
    
trainingData = []
trainingDataTweet = []
combinedDataTrain = []
with open("train.data.jsonl", encoding="utf8") as f:
    coreIndex = 0
    for line in f:
        row = json.loads(line)
        trainingData.append(row)
        text = ""
        for tweet in row:
            text += " " + tweet["text"]
        combinedDataTrain.append(text[1:])
        trainingDataTweet.append(row[0]["text"])
      
# Development Data    
devLabels = {}
with open("dev.label.json",  encoding="utf8") as f:
    devLabels = json.load(f)    
    
devData = []
devDataTweet = []
combinedDataDev = []
with open("dev.data.jsonl", encoding="utf8") as f:
    coreIndex = 0
    for line in f:
        row = json.loads(line)
        devData.append(row)
        
        text = ""
        for tweet in row:
            text += " " + tweet["text"]
        combinedDataDev.append(text[1:])
        devDataTweet.append(row[0]["text"])

# Test Data
testData = []
testDataTweet = []
combinedDataTest = []
with open("test.data.jsonl", encoding="utf8") as f:
    coreIndex = 0
    for line in f:
        row = json.loads(line)
        testData.append(row)
        
        text = ""
        for tweet in row:
            text += " " + tweet["text"]
        combinedDataTest.append(text[1:])
        testDataTweet.append(row[0]["text"])
        
print (f"Train Labels: {len(trainLabels)}, Train Data: {len(trainingData)}")
print (f"Train Labels: {len(devLabels)}, Train Data: {len(devData)}")
print (f"Train Labels: {len(testData)}")

Train Labels: 4641, Train Data: 4641
Train Labels: 580, Train Data: 580
Train Labels: 581


In [13]:
testData[234][0]["user"]["id"]

2190056023

In [3]:
import numpy as np
y_train = list(trainLabels.values())
y_dev = list(devLabels.values())

In [4]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(combinedDataTrain)

x_train = tokenizer.texts_to_matrix(combinedDataTrain, mode="count") #BOW representation
x_dev = tokenizer.texts_to_matrix(combinedDataDev, mode="count") #BOW representation
x_test = tokenizer.texts_to_matrix(combinedDataTest, mode="count") #BOW representation

vocab_size = x_train.shape[1]

In [ ]:
#tokenise the input into word sequences

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(trainingDataTweet)

xseq_train = tokenizer.texts_to_sequences(combinedDataTrain)
xseq_dev = tokenizer.texts_to_sequences(combinedDataDev)
xseq_test = tokenizer.texts_to_sequences(combinedDataTest)

len(xseq_train[0])

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 500
xseq_train = pad_sequences(xseq_train, padding='post', maxlen=maxlen)
xseq_dev = pad_sequences(xseq_dev, padding='post', maxlen=maxlen)
xseq_test = pad_sequences(xseq_test, padding='post', maxlen=maxlen)
print(xseq_train[0])

In [5]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
score = classifier.score(x_dev, y_dev)

print("Accuracy:", score)

c:\users\sharo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy: 0.8379310344827586


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(['non-rumour', 'rumour'])
y_train_binary = le.transform(y_train)
y_dev_binary = le.transform(y_dev)

In [ ]:
from keras.layers import LSTM
from keras.models import Sequential
from keras import layers

embedding_dim = 10

#word order preserved with this architecture
model3 = Sequential(name="lstm")
model3.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model3.add(LSTM(10))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model3.summary()

In [ ]:
model3.fit(xseq_train, y_train_binary, epochs=20, verbose=True, validation_data=(xseq_dev, y_dev_binary), batch_size=10)

loss, accuracy = model3.evaluate(xseq_dev, y_dev_binary, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
y_test_LSTM = model3.predict(xseq_test)
y_test_LSTM = list(map(lambda label : int(round(label[0])), y_test_LSTM))
y_predict_LSTM = le.inverse_transform(y_test_LSTM)


data = {}

for i in range(len(testData)):
    data[testData[i][0]['id']] = y_predict_LSTM[i]

with open('test-output.json', 'w') as outfile:
    json.dump(data, outfile)